In [ ]:
#llm RAG 플러그인 설정(임베딩)

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 필요한 라이브러리 import 수정
import pandas as pd
from langchain_core.documents import Document # LangChain의 기본 문서 형식을 import
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:
def setup_rag_pipeline_chroma(file_path: str, embedding_model_name: str, persist_directory: str):
    """
    CSV 파일로부터 ChromaDB RAG 파이프라인을 구축하고 검색기(Retriever)를 반환합니다.
    """
    # 1. Embed: 임베딩 모델 로드 (이전과 동일)
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    print("✅ 임베딩 모델을 로드했습니다.")
    
    # 2. 벡터 스토어 생성 또는 로드
    # persist_directory에 DB가 이미 존재하면 새로 만들지 않고 바로 로드합니다.
    if os.path.exists(persist_directory):
        print("✅ 기존 ChromaDB를 로드합니다.")
        vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    else:
        print("⚠️ 기존 ChromaDB가 없으므로 새로 생성합니다.")
        # 1. Pandas로 CSV 파일 불러오기
        df = pd.read_csv(file_path, index_col=0)
        print(f"✅ Pandas로 CSV 파일을 불러왔습니다. (총 {len(df)}개 행)")

        # 2. 각 행을 자연어 문장으로 변환하고, LangChain 'Document' 객체로 만들기
        langchain_documents = []
        for timestamp, values in df.iterrows():
            # 직접 만드신 자연어 변환 로직
            sentence = f"'{timestamp}'의 "
            value_strings = [f"'{col}'는 {val}" for col, val in values.items()]
            sentence += ", ".join(value_strings)
            
            # 변환된 문장을 page_content로 하는 Document 객체 생성
            # metadata에 원본 시간 정보를 넣어두면 나중에 유용하게 사용할 수 있습니다.
            doc = Document(
                page_content=sentence,
                metadata={"timestamp": timestamp}
            )
            langchain_documents.append(doc)
        
        print(f"✅ {len(langchain_documents)}개의 자연어 문서를 생성했습니다.")
        
        # Store: ChromaDB를 생성하고 파일로 저장(persist)합니다.
        vectorstore = Chroma.from_documents(
            documents=langchain_documents, 
            embedding=embeddings,
            persist_directory=persist_directory # 이 경로에 DB 파일이 저장됩니다.
        )
        print(f"✅ ChromaDB를 생성하고 '{persist_directory}' 경로에 저장했습니다.")

    # 3. Retriever 생성 (이전과 완벽히 동일)
    retriever = vectorstore.as_retriever()
    print("✅ Retriever를 생성했습니다. RAG 파이프라인 준비 완료.")
    
    return retriever

In [ ]:
# --- RAG 파이프라인 실행 ---
CSV_FILE_PATH = 'RTDB_test.csv'
EMBEDDING_MODEL = 'nlpai-lab/KURE-v1'
CHROMA_DB_PATH = 'chroma_db' # DB 파일이 저장될 폴더 이름
# ChromaDB용 함수를 호출합니다.
rag_retriever = setup_rag_pipeline_chroma(CSV_FILE_PATH, EMBEDDING_MODEL, CHROMA_DB_PATH)

In [ ]:
#llm 메인 모델 설정

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="GEMINI_API_KEY",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [ ]:
from llama_cpp import Llama
import json
import os
from llama_cpp.llama_chat_format import Jinja2ChatFormatter

In [ ]:
# --- 2. Jinja 채팅 템플릿 파일에서 로드 (수정된 부분) ---
# 'lmalign_v1.jinja' 파일이 이 스크립트와 같은 디렉토리에 있는지 확인하세요.
jinja_template_path = "lmalign_v1.jinja"

if not os.path.exists(jinja_template_path):
    raise FileNotFoundError(
        f"'{jinja_template_path}' 파일을 찾을 수 없습니다. "
        "스크립트와 동일한 디렉토리에 파일을 위치시켜 주세요."
    )

with open(jinja_template_path, "r", encoding="utf-8") as f:
    jinja_template = f.read()

# --- 3. Jinja2ChatFormatter 인스턴스 생성 ---
chat_handler = Jinja2ChatFormatter(
    template=jinja_template,
    eos_token="<|eot_id|>",
    bos_token="<|begin_of_text|>"
)

In [ ]:
# --- 3. Llama 모델 로드 ---
# 다운로드한 모델과 파일에서 읽어온 Jinja 템플릿을 사용하여 모델을 초기화합니다.
llm = Llama(
  model_path=model_path,
  n_ctx=8192,         # Context window 사이즈
  n_gpu_layers=-1,    # GPU 가속 사용 (가능한 경우)
  verbose=False,
)

In [ ]:
# --- 4. 추론 실행 ---
# 서비스 설명 및 출력 통제 지시사항을 포함한 시스템 프롬프트 구성
#추가 프롬: - 반드시 JSON 포맷으로 응답

## 평가자 llm 시스템 프롬프트
router_system_prompt = """
당신은 사용자가 특허를 검색할 수 있도록 돕는 'AI 특허 전략가'입니다.

당신의 첫 번째 임무는 사용자의 [초기 아이디어]를 분석하여, 이 아이디어가 유의미한 특허 검색을 수행하기에 '충분히 구체적인지' 아니면 '너무 광범위한지' 판단하는 것입니다.

당신은 두 가지 시나리오 중 하나로만 응답해야 합니다.

---
### 📜 시나리오 1: 아이디어가 '너무 광범위한' 경우 (예: "자동차", "AI", "좋은 카메라")

만약 아이디어가 일반적인 용어이거나, 구체적인 기술적 구성 또는 해결하려는 문제가 명확하지 않다면, **당신은 절대 검색을 시작해서는 안 됩니다.**

대신, 다음과 같은 3단계로 사용자에게 **'아이디어 구체화'**를 요청하는 응답을 생성해야 합니다.

1.  **[문제 지적]** 아이디어가 훌륭한 시작점이지만, 현재 상태로는 유의미한 특허 검색이 어렵다는 것을 부드럽게 설명합니다. (예: "아이디어가 너무 광범위하여 관련된 특허가 수천 개 이상일 수 있습니다.")
2.  **[개방형 질문]** 사용자가 아이디어를 구체화할 수 있도록 핵심적인 질문을 1~2개 합니다. (예: "혹시 어떤 '문제'를 해결하는 아이디어인가요?", "기존 기술과 다른 '핵심 기능'은 무엇인가요?")
3.  **[키워드 제안]** 사용자의 생각을 자극할 수 있도록, 입력된 광범위한 키워드와 관련된 **'구체적인 기술 분야 키워드' 3~4개**를 제안합니다.

**[예시 응답 (사용자 입력: "자동차")]**
> "자동차'는 흥미로운 주제네요! 다만, '자동차'만으로는 검색 범위가 너무 넓어서 유의미한 유사 특허를 찾기 어렵습니다.
> 
> 혹시 생각하고 계신 구체적인 기술 분야가 있으신가요? 예를 들어 어떤 **문제**를 해결하거나 **개선**하고 싶은 부분이 있으신가요?
> 
> 참고로, 자동차 관련 특허는 보통 다음과 같은 구체적인 분야로 나뉩니다.
> * 전기차 배터리 냉각 시스템
> * 자율주행용 라이다(LiDAR) 센서
> * 차량 내부 인포테인먼트 UI
> * 충돌 방지용 능동형 안전장치
> 
> 조금 더 구체적으로 알려주시면, 제가 더 정확하게 분석해 드릴 수 있습니다."

---
### 📜 시나리오 2: 아이디어가 '충분히 구체적인' 경우

만약 아이디어가 **'특정 기술', '해결하려는 문제', '독창적인 구성'** 중 하나 이상을 명확히 포함하고 있다면 (예: "스마트폰 카메라로 피부 상태를 진단하고 AI로 화장품을 추천하는 앱"), 아이디어를 칭찬하고 **즉시 다음 단계(검색)로 넘어갈 것임**을 사용자에게 알려야 합니다.

**[매우 중요]** 이 경우, 응답의 **맨 마지막 줄**에 **`[SEARCH_READY]`** 태그를 포함해야 합니다. (이 태그는 백엔드 시스템이 실제 RAG 검색을 트리거하는 신호로 사용됩니다.)

**[예시 응답 (사용자 입력: "AI 기반 피부 진단 앱")]**
> "매우 흥미롭고 구체적인 아이디어입니다. 'AI', '피부 진단', '스마트폰 카메라' 등 핵심 키워드가 명확하여 바로 유사 특허 검색을 진행할 수 있겠습니다.
> 
> 지금부터 입력하신 아이디어를 기반으로 RAG 및 LLM-as-Judge를 통해 유사 특허 분석을 시작하겠습니다. 잠시만 기다려주세요.
> 
> [SEARCH_READY]"
"""

evaluation_system_prompt = """
당신은 AI 검색 결과를 검증하는 '최종 특허 심사관'입니다.

[특허 문서 조각]은 1차 AI 검색(RAG)을 통해 [사용자 아이디어]와 '의미상' 유사할 가능성이 있다고 **후보로** 검색된 결과입니다.

당신의 임무는 이 1차 검색 결과를 맹신하지 말고, [사용자 아이디어]의 **'기술적 핵심 원리'** 관점에서 **전문가로서 2차 검증(Evaluation)**을 수행하는 것입니다. 1차 검색이 표면적인 키워드나 문맥에 속았을 수 있으니(예: 'AI 진단'은 같으나 '피부' vs '자동차 도색'처럼 분야가 다름), 이를 비판적으로 분석해야 합니다.

당신의 응답은 `evaluation_idea` 함수를 호출하는 것이어야 합니다.

---
### ⚖️ 평가 지침

1.  **`eval_score` (0-100점 사이의 정수):**
    * 이 점수는 [아이디어]와 [특허 조각]의 **'기술적 핵심 원리의 일치도'**입니다.
    * 아래 벤치마크를 **참고**하여 구체적인 정수 점수를 도출하세요.

    * **0점 (기준):** 1차 검색이 명백히 오류임. 키워드만 겹칠 뿐, 기술 분야와 원리가 완전히 무관합니다.
    * **25점 (참고 기준):** 1차 검색이 '의미'는 맞췄으나(예: 'AI 진단'), **'기술 분야(Domain)'**가 다릅니다. (예: 아이디어는 '의료 피부'인데 특허는 '자동차 도색')
    * **50점 (중간 기준):** 기술 분야와 해결하려는 문제는 유사하나, **'핵심 기술 수단(접근 방식)'**이 상이합니다.
    * **75점 (참고 기준):** 핵심 기술 원리 및 접근 방식이 매우 유사하며, 일부 부수적인 구성 요소에서만 차이를 보입니다.
    * **100점 (기준):** 두 내용의 핵심 기술적 사상이 실질적으로 동일합니다.

2.  **`reason` (문자열):**
    * **[매우 중요]** 이 점수를 준 '이유'를 명확히 설명해야 합니다.
    * **(점수가 낮을 경우):** "1차 검색에서는 유사하게 보였으나, [아이디어]의 '핵심 분야(예: 의료)'와 달리 이 특허는 '(예: 자동차)' 분야에 관한 것이므로 점수를 낮게 책정했습니다."와 같이, RAG가 왜 이것을 가져왔는지 + 그럼에도 왜 점수가 낮은지를 함께 설명해야 합니다.
    * **(점수가 높을 경우):** [아이디어]의 '[X] 개념'이 특허의 '[Y] 부분'과 어떻게 기술적으로 일치하는지 설명합니다.
---

이제 [사용자 아이디어]와 [특허 문서 조각]을 바탕으로 분석을 수행하고, 즉시 `evaluation_idea` 함수를 호출하세요.
"""


In [ ]:
# --- 5. Function Calling 테스트 ---

# 5-2. 도구(함수) 목록 정의 (OpenAI tool-call 형식)
eval_tools = [
    {
        "type": "function",
        "function": {
            "name": "evaluation_idea",
            "description": "사용자의 아이디어와 특허 문서 조각의 유사도를 분석하여, 0-100점 사이의 점수와 그 근거를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "eval_score": {
                        "type": "integer",
                        "description": "사용자 아이디어와 특허 조각 간의 기술적 유사도 점수. 0 (완전히 무관함)에서 100 (기술적으로 동일함) 사이의 정수입니다.",
                        "minimum": 0,
                        "maximum": 100
                    },
                    "reason": {
                        "type": "string",
                        "description": "해당 점수를 부여한 구체적인 이유. 특허 조각의 어느 부분이 아이디어의 어떤 개념과 유사한지(또는 다른지) 명확히 짚어서 설명해야 합니다."
                    }
                },
                "required": ["eval_score", "reason"],
            },
        },
    }
]



In [ ]:
import re
import json

def execute_router(user_query: str, gemini_model_name: str = "gemini-pro"):
    """
    사용자 아이디어를 받아 게이트키퍼 LLM을 호출하고,
    결과에 따라 RAG 검색을 트리거하거나 사용자에게 피드백을 반환합니다.
    """
    
    if not client:
        print("LLM 클라이언트가 초기화되지 않았습니다. 함수 실행을 중단합니다.")
        return {"status": "error", "message": "LLM client not initialized."}

    print(f"\n--- [EXECUTE ROUTER] ---")
    print(f"입력 아이디어: '{user_query}'")

    try:
        # 1. '아이디어 게이트키퍼' LLM 호출
        response = client.chat.completions.create(
            model=gemini_model_name,  # OpenAI 호환 모드로 사용할 Gemini 모델명
            messages=[
                {"role": "system", "content": router_system_prompt},
                {"role": "user", "content": user_query}
            ],
            temperature=0.3  # 라우팅 작업은 일관성 있게 0으로 설정
        )

        llm_feedback = response.choices[0].message.content
        print("\n[게이트키퍼 LLM 응답]")
        print("--------------------")
        print(llm_feedback)
        print("--------------------")

        # 2. [SEARCH_READY] 태그 확인
        if "[SEARCH_READY]" in llm_feedback:
            # 시나리오 2: 검색 준비 완료
            print("\n[라우터 결정] 'SEARCH_READY' 태그 확인. RAG 검색을 시작합니다.")
            
            # RAG 리트리버 호출
            search_results = rag_retriever.invoke(user_query)
            
            # 이 결과(search_results)를 다음 단계(LLM-as-Judge)로 넘깁니다.
            return {
                "status": "search_triggered",
                "message": llm_feedback, # "분석을 시작합니다..." 메시지
                "search_results": search_results # RAG 검색 결과
            }
        else:
            # 시나리오 1: 아이디어 구체화 필요
            print("\n[라우터 결정] 'SEARCH_READY' 태그 없음. 사용자에게 피드백을 반환합니다.")
            
            # 이 피드백(llm_feedback)을 사용자에게 보여줍니다.
            return {
                "status": "needs_refinement",
                "message": llm_feedback, # "아이디어를 더 구체화해주세요..." 메시지
                 "search_results": None
            }

    except Exception as e:
        print(f"\n--- [오류] LLM API 호출 또는 라우팅 중 오류 발생 ---")
        print(f"에러 상세: {e}")
        return {"status": "error", "message": str(e)}
        
import pprint
from typing import List, Dict, Any

# 이 함수는 다음 대화에서 실제 LLM API를 호출하도록 구현할 것입니다.
# (Gemini API + 'evaluation_idea' 툴 호출 로직)
# --- 3. [핵심 구현] llm_eval 함수 ---

def llm_eval(chunk_text: str, user_idea: str, gemini_model_name: str = "gemini-pro") -> Dict[str, Any]:
    """
    하나의 특허 조각(chunk_text)과 사용자 아이디어(user_idea)를 받아
    LLM-as-Judge(Gemini API + Tool Calling)를 수행하고, 점수와 이유를 반환합니다.
    """
    
    if not client:
        print("  [LLM_EVAL] 오류: LLM 클라이언트가 초기화되지 않았습니다.")
        return {"eval_score": -1, "reason": "LLM client not initialized."}

    # 1. LLM에 전달할 User Message 구성
    # [중요] 시스템 프롬프트가 '아이디어'와 '조각'을 구분할 수 있도록 명확히 전달
    user_message_content = f"""
    [사용자 아이디어]:
    {user_idea}
    
    [특허 문서 조각]:
    {chunk_text}
    """
    
    print(f"  [LLM_EVAL] API 호출 시작... (Model: {gemini_model_name})")

    try:
        # 2. Gemini API (OpenAI 호환 모드) 호출
        response = client.chat.completions.create(
            model=gemini_model_name,
            messages=[
                {"role": "system", "content": evaluation_system_prompt},
                {"role": "user", "content": user_message_content}
            ],
            tools=eval_tools,
            # [중요] LLM이 텍스트로 응답하지 않고 반드시 'evaluation_idea' 툴을 호출하도록 강제
            tool_choice={"type": "function", "function": {"name": "evaluation_idea"}}
        )

        # 3. LLM의 툴 호출(Tool Call) 응답 파싱
        message = response.choices[0].message
        
        if message.tool_calls:
            tool_call = message.tool_calls[0]
            
            # 4. 'evaluation_idea' 함수가 맞는지 확인
            if tool_call.function.name == "evaluation_idea":
                # arguments는 JSON '문자열'이므로, 'dict'로 파싱
                arguments = json.loads(tool_call.function.arguments)
                
                eval_score = arguments.get("eval_score")
                reason = arguments.get("reason")

                if eval_score is not None and reason is not None:
                    # 5. 성공: 점수와 이유 반환
                    print(f"  [LLM_EVAL] 성공: 점수={eval_score}")
                    return {"eval_score": eval_score, "reason": reason}
                else:
                    print("  [LLM_EVAL] 오류: LLM이 툴을 호출했으나, 'eval_score' 또는 'reason' 필드가 누락되었습니다.")
                    return {"eval_score": -1, "reason": "LLM output parsing error (missing fields)."}
            else:
                # (일어날 가능성 낮음)
                print(f"  [LLM_EVAL] 오류: LLM이 예상치 못한 툴 '{tool_call.function.name}'을 호출했습니다.")
                return {"eval_score": -1, "reason": "LLM called unexpected tool."}
        
        else:
            # (tool_choice로 강제했기 때문에 일어날 가능성 낮음)
            print("  [LLM_EVAL] 오류: LLM이 툴을 호출하지 않고 텍스트로 응답했습니다.")
            print(f"    LLM 응답: {message.content}")
            return {"eval_score": -1, "reason": "LLM failed to call the required tool."}

    except Exception as e:
        # API 키 오류, 네트워크 오류, 타임아웃 등
        print(f"  [LLM_EVAL] API 호출 중 심각한 오류 발생: {e}")
        return {"eval_score": -1, "reason": f"LLM API call failed: {str(e)}"}

def result_evaluation(search_results: List[Dict[str, Any]], user_idea: str) -> List[Dict[str, Any]]:
    """
    RAG 검색 결과(search_results) 리스트와 사용자 아이디어를 받아
    각 조각을 LLM으로 평가하고, 최종 결과 리스트를 반환합니다.
    
    Args:
        search_results: execute_router에서 반환된 ChromaDB 검색 결과 리스트.
                        (예: [ {'page_content': '...', 'metadata': {...}}, ... ])
        user_idea:      게이트키퍼를 통과한 원본 사용자 아이디어.

    Returns:
        최종 분석 결과 리스트.
        (예: [ {'eval_score': 90, 'reason': '...', 'metadata': {...}}, ... ])
    """
    
    # 최종 결과물 (딕셔너리의 리스트)
    analyzed_patent_list: List[Dict[str, Any]] = []
    
    print(f"\n--- [RESULT EVALUATION] ---")
    print(f"총 {len(search_results)}개의 검색된 조각(chunk)에 대해 LLM-as-Judge 평가를 시작합니다.")

    # ChromaDB (또는 LangChain) 리트리버는 보통
    # [{'page_content': '...', 'metadata': {...}}, ...] 형태의 리스트를 반환합니다.
    
    for i, chunk_document in enumerate(search_results):
        try:
            # 1. 메타데이터와 청크 텍스트 분리
            chunk_text = chunk_document.get('page_content')
            metadata = chunk_document.get('metadata')
            
            if not chunk_text or metadata is None:
                print(f"  [경고] {i+1}번째 조각의 형식이 잘못되었습니다. (page_content 또는 metadata 누락) - 건너뜁니다.")
                continue

            # 2. llm_eval 호출 (LLM-as-Judge 실행)
            # [중요] user_idea를 반드시 함께 전달합니다.
            analysis_info = llm_eval(chunk_text, user_idea)
            
            # 3. 분석 정보(eval_score, reason)와 원본 메타데이터를 하나로 결합
            
            final_result_item = {
                "eval_score": analysis_info.get("eval_score"),
                "reason": analysis_info.get("reason"),
                "metadata": metadata  # 원본 메타데이터 덩어리(dict)를 통째로 포함
            }
            
            # 4. 최종 리스트에 추가
            analyzed_patent_list.append(final_result_item)
            
            print(f"  [완료] {i+1}/{len(search_results)} 번째 조각 평가 완료. (점수: {final_result_item['eval_score']})")

        except Exception as e:
            print(f"  [오류] {i+1}번째 조각 평가 중 오류 발생: {e}")
            # 오류가 발생한 청크는 건너뛰고 계속 진행
            continue

    print(f"--- [RESULT EVALUATION] ---")
    print(f"총 {len(analyzed_patent_list)}개의 조각에 대한 평가가 완료되었습니다.")
    
    # 5. 최종 결과 리스트 반환
    return analyzed_patent_list

In [ ]:
def answer_with_rag(user_query: str):
    """
    사용자 질문에 대해 RAG를 통해 찾은 정보와 함께 LLM에게 답변을 요청하는 함수
    """
    print(f"\n--- 사용자 질문: {user_query} ---")
    
    # 1. Retrieve: RAG Retriever로 사용자 질문과 가장 관련 높은 문서를 검색합니다.
    retrieved_docs = rag_retriever.invoke(user_query)
    print(f"📚 {len(retrieved_docs)}개의 관련 문서를 찾았습니다.")

    # 2. 컨텍스트(Context) 생성: 검색된 문서들의 내용을 하나의 문자열로 합칩니다.
    context_data = "\n".join([f"- {doc.page_content}" for doc in retrieved_docs])
    '''
    print("--- 검색된 RAG 데이터 ---")
    print(context_data)
    print("-------------------------")
    '''
    # 3. 프롬프트 구성: 기존의 프롬프트 형식에 질문과 RAG 데이터를 결합합니다.
    # 이는 get_single_turn_response 함수 내부에서 처리되므로,
    # 해당 함수에 전달할 형태로 질문을 재구성합니다.
    final_query_for_llm = f"""
    아래 [데이터]를 참고하여 질문에 대답해주세요.
    
    [질문]
    {user_query}

    [데이터]
    {context_data}
    """
    
    # 4. Generate: 재구성된 질문을 기존에 만들었던 LLM 호출 함수에 전달합니다.
    final_answer = get_single_turn_response(final_query_for_llm)
    
    return final_answer

In [ ]:
# --- 실제 서비스 로직 예시 (Stateless) ---

# Case 1: 일반적인 파싱
print("--- Case 1: 함수 호출 질문 ---")
user_input_1 = ""
final_response_1 = answer_with_rag(user_input_1)
print("\n[최종 답변]")
print(final_response_1)